this code for google docs . google cloud API for automatically storing the images genearted into google docs

In [ ]:
!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 19.3 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.151.0
    Uninstalling google-api-python-client-2.151.0:
      Successfully uninstalled google-api-python-client-2.151.0


authenticate and authorise the google acount

In [1]:
from google.colab import auth
auth.authenticate_user()

from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaInMemoryUpload

creds, _ = default()
drive_service = build('drive', 'v3', credentials=creds)
docs_service = build('docs', 'v1', credentials=creds)


create new google doc

In [2]:
doc = docs_service.documents().create(body={"title": "My Document"}).execute()
doc_id = doc['documentId']


generate and upload image to google doc

In [3]:
import io
import matplotlib.pyplot as plt

def generate_image():
    fig, ax = plt.subplots()
    ax.plot([1, 2, 3, 4], [1, 4, 2, 3])
    ax.set_title("Sample Image")

    image_buf = io.BytesIO()
    plt.savefig(image_buf, format='png')
    plt.close(fig)
    image_buf.seek(0)

    return image_buf


def upload_image_to_drive(image_buf):
    file_metadata = {'name': 'my_image.png', 'mimeType': 'image/png'}
    media = MediaInMemoryUpload(image_buf.getvalue(), mimetype='image/png')
    file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()

     #Make the uploaded file publicly accessible
    permission = {
        'type': 'anyone',
        'role': 'reader'
    }
    drive_service.permissions().create(fileId=file.get('id'), body=permission).execute()


    return file.get('id')



def insert_image_into_doc(doc_id, file_id):
    file_url = f"https://drive.google.com/uc?id={file_id}"
    requests = [
        {
            'insertInlineImage': {
                'location': {
                    'index': 1,  # Change index based on where you want to insert
                },
                'uri': file_url,
                'objectSize': {
                    'height': {
                        'magnitude': 400,
                        'unit': 'PT',
                    },
                    'width': {
                        'magnitude': 400,
                        'unit': 'PT',
                    },
                },
            }
        }
    ]
    docs_service.documents().batchUpdate(documentId=doc_id, body={'requests': requests}).execute()


      # Delete the image from Drive after inserting
   # drive_service.files().delete(fileId=file_id).execute()
    print(f"Image with ID {file_id} deleted from Drive.")

# Example of generating, uploading, and inserting multiple images
for _ in range(3):  # Adjust the range for the number of images you want to generate
    image_buf = generate_image()
    file_id = upload_image_to_drive(image_buf)
    insert_image_into_doc(doc_id, file_id)


Image with ID 1w5tu6YyP0YporLgJvdISKkFa6Xr8klZy deleted from Drive.
Image with ID 10IV-is5aB0H0TrHuaFAyMv92nrgyhjaL deleted from Drive.
Image with ID 1iDJuSuFbqM3wQ_6vDHp3kR91Y-aKz2_D deleted from Drive.
